In [14]:
%pip install newsapi-python

Note: you may need to restart the kernel to use updated packages.


In [15]:
import json
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from newsapi import NewsApiClient
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import openai
import torch

In [16]:
# API configuration
openai.api_type = "azure"
openai.api_base = "https://gpt-722.openai.azure.com/"
openai.api_key = "bbdc656728034b52870b2618e2cb36cb"
openai.api_version = "2023-09-01-preview"
MODEL = "GPT-4-for-EDU"
news_api_key = "f8f1f4e2c57a453fa84c593c3aff7494"
newsapi = NewsApiClient(api_key=news_api_key)

In [17]:
# Get all useful sources
sources = newsapi.get_sources(language='en')
sources = sources['sources']
sources = pd.DataFrame(sources)
sources

,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
1,abc-news-au,ABC News (AU),"Australia's most trusted source of local, nati...",http://www.abc.net.au/news,general,en,au
2,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",http://www.aljazeera.com,general,en,us
3,ars-technica,Ars Technica,The PC enthusiast's resource. Power users and ...,http://arstechnica.com,technology,en,us
4,associated-press,Associated Press,The AP delivers in-depth coverage on the inter...,https://apnews.com/,general,en,us
...,...,...,...,...,...,...,...
76,the-washington-times,The Washington Times,The Washington Times delivers breaking news an...,https://www.washingtontimes.com/,general,en,us
77,time,Time,Breaking news and analysis from TIME.com. Poli...,http://time.com,general,en,us
78,usa-today,USA Today,"Get the latest national, international, and po...",http://www.usatoday.com/news,general,en,us
79,vice-news,Vice News,"Vice News is Vice Media, Inc.'s current affair...",https://news.vice.com,general,en,us


In [18]:
# Concatenate the sources into a string for api call
listOfSources = ""
for i in range(len(sources)):
    listOfSources += f'{sources["id"][i]}, '
# results = newsapi.get_everything(q='+ETF AND (launch OR list OR delist)',language='en',sources=listOfSources,sort_by='publishedAt',from_param='2023-10-05', to='2023-11-04')
# results = results['articles']
# results

In [19]:
# Function to extract news articles from the results into a txt file for further processing
# def extract_content(url):
#     driver = webdriver.Chrome()
#     driver.get(url)
#     title = driver.title
#     content = driver.find_elements('xpath', '//script[]')
#     content_list = []
#     for i in content:
#         content_list.append(i.text)
#     content_list = " ".join(content_list)
#     driver.close()
#     return [title, content_list]

In [20]:
# Get today's date and the date 31 days ago
from datetime import datetime, timedelta
endtime = datetime.today().strftime('%Y-%m-%d')
starttime = (datetime.today() - timedelta(days=31)).strftime('%Y-%m-%d')
print(starttime, endtime)

2023-10-09 2023-11-09


In [21]:
# finbert query function
def finbert_query(text):
    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

    result = nlp(text)
    return result[0].get('label')

# GPT summary function
def gpt_summary(title, description, content):
    response = openai.ChatCompletion.create(
        engine=MODEL,
        messages=[
            {"role": "system", "content": "You are a financial analyst. You are reading a news article about a recent ETF launch. You need to summarize the article and write a short report about it for investment specialists and executives in JPMC's ETF sales team."},
            {"role": "user", "content": f"Title: {title}"},
            {"role": "user", "content": f"Description: {description}"},
            {"role": "user", "content": f"Description: {content}"}
        ],
        temperature=0.2,
    )
    result = response['choices'][0]['message']['content']
    print(result)
    return result

In [22]:
# API call for ETF launch news, newsapi library does not support searching only title
launching_result = requests.get(f'https://newsapi.org/v2/everything/?q=ETF AND (launch OR list OR delist)&sources={listOfSources}&searchIn=title,description&language=en&sortBy=publishedAt&from={starttime}&to={endtime}&apiKey={news_api_key}')
launching_result = launching_result.json()
export_result = dict(sorted(launching_result.items()))
export_result = json.dumps(launching_result, indent=4)

# Export the results to a json file
with open('launch_related_news.json', 'w') as f:
    f.write(export_result)

num_results = launching_result['totalResults']
launching_result = launching_result['articles']
launching_result = pd.DataFrame(launching_result)
launching_result = launching_result.drop_duplicates(subset=['title'])
launching_result = launching_result.reset_index(drop=True)
launching_result = launching_result.drop(columns=['source', 'author', 'url', 'urlToImage', 'publishedAt'])
launching_analysis_list = []
launching_analysis = pd.DataFrame(columns=['title', 'sentiment', 'summary'], index=range(num_results))
for i in range(num_results):
    temp = launching_result.iloc[i]
    launching_analysis_list.append(temp['title']+'\n' + temp['description']+ '\n'+ temp['content'])
    launching_analysis.iloc[i] = [temp['title'], finbert_query(launching_analysis_list[i]), gpt_summary(temp['title'],temp['description'],temp['content'])]
launching_analysis


Summary:

The recent news of BlackRock Inc. potentially receiving approval to launch a spot Exchange-Traded Fund (ETF) led to a 10% surge in Bitcoin's value, reaching $30,002, its highest since March. However, the spike was short-lived as BlackRock clarified that its application is still under review. This incident provided traders with an insight into the potential impact of the US Securities & Exchange Commission's (SEC) upcoming decision on the approval of ETFs that directly invest in Bitcoin. 

This event suggests that the launch of a Bitcoin ETF could significantly influence the cryptocurrency's price. It also highlights the market's sensitivity to regulatory decisions, particularly those related to digital assets. The SEC's decision on Bitcoin ETFs is highly anticipated and could potentially reshape the digital asset landscape. 

For JPMC's ETF sales team, this development underscores the importance of closely monitoring regulatory decisions and market reactions. It also suggests

,title,sentiment,summary
0,"Bitcoin’s 10% jump to $30,000 hints at trader ...",Negative,Summary:\n\nThe recent news of BlackRock Inc. ...
1,"Bitcoin jumps 10%, then slides after BlackRock...",Positive,Summary:\n\nThe article discusses the recent f...


In [27]:
# Export the results to a json file
launching_summary = launching_analysis.to_dict(orient='records')
result = {}
with open('launch_related_analyzation.json', 'w') as f:
    for i in range(len(launching_summary)):
        result[i] = launching_summary[i]
    result = dict(result.items())
    f.write(json.dumps(result, indent=4))


In [24]:
# API Call for ETF regulatory news
regulatory_result = requests.get(f'https://newsapi.org/v2/everything/?q=ETF AND (regulatory OR regulation)&sources={listOfSources}&searchIn=title,description&language=en&sortBy=publishedAt&from={starttime}&to={endtime}&apiKey={news_api_key}')
regulatory_result = regulatory_result.json()
export_result = dict(sorted(regulatory_result.items()))
export_result = json.dumps(regulatory_result, indent=4)

# Export the results to a json file
with open('regulatory_related_news.json', 'w') as f:
    f.write(export_result)

num_regulatory_results = regulatory_result['totalResults']
regulatory_result = regulatory_result['articles']
regulatory_result = pd.DataFrame(regulatory_result)
regulatory_result = regulatory_result.drop_duplicates(subset=['title'])
regulatory_result = regulatory_result.reset_index(drop=True)
regulatory_result = regulatory_result.drop(columns=['source', 'author', 'url', 'urlToImage', 'publishedAt'])
regulatory_analysis_list = []
regulatory_analysis = pd.DataFrame(columns=['title', 'sentiment', 'summary'], index=range(num_regulatory_results))
for i in range(num_regulatory_results):
    temp = regulatory_result.iloc[i]
    regulatory_analysis_list.append(temp['title']+'\n' + temp['description']+ '\n'+ temp['content'])
    regulatory_analysis.iloc[i] = [temp['title'], finbert_query(regulatory_analysis_list[i]), gpt_summary(temp['title'],temp['description'],temp['content'])]

Summary: 

BlackRock, the world's largest asset manager, has proposed a Bitcoin exchange-traded fund (ETF) that was added to a clearing-house eligibility file in August. However, this move does not imply any regulatory approval. The ETF is still under review by the regulatory authorities. The addition of the ETF to the eligibility file is a procedural step that allows the Depository Trust & Clearing Corporation (DTCC) to prepare for the potential launch of the product. The news comes amidst increasing interest in cryptocurrency ETFs, with several other firms also filing for similar products. Further updates on the status of BlackRock's Bitcoin ETF will be closely monitored.


In [28]:
# Export the results to a json file
regulatory_summary = regulatory_analysis.to_dict(orient='records')
result = {}
with open('regulatory_related_analyzation.json', 'w') as f:
    for i in range(len(regulatory_summary)):
        result[i] = regulatory_summary[i]
    result = dict(result.items())
    f.write(json.dumps(result, indent=4))